# Structured and time series data

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
from pandas.io.json import json_normalize
np.set_printoptions(threshold=50, edgeitems=20)

PATH='/home/borowis/s3/'

/home/borowis/.pyenv/versions/3.6.6/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
def load_df(csv_path, json_columns=JSON_COLUMNS):

    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in json_columns}, 
                     dtype={'fullVisitorId': 'str'})
    
    for column in json_columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    return df

In [4]:
table_names = ['train', 'test']

In [ ]:
tables = [load_df(f'{PATH}{fname}.csv') for fname in table_names]

In [ ]:
from IPython.display import HTML, display

In [ ]:
for t in tables: display(t.head())

In [ ]:
for t in tables: display(DataFrameSummary(t).summary())

## Data Cleaning / Feature Engineering

As a structured data problem, we necessarily have to go through all the cleaning and feature engineering, even though we're using a neural network.

In [ ]:
train, test = tables

In [ ]:
len(train),len(test)

Drop non-unique columns

In [ ]:
cols_to_drop = [c for c in train.columns if train[c].nunique()==1]; cols_to_drop

In [ ]:
train.drop(cols_to_drop, axis=1, inplace=True)
test.drop([col for col in cols_to_drop if col in test.columns], axis=1, inplace=True)

Columns in test set missing from train 

In [ ]:
print("Variables not in test but in train : ", set(train.columns).difference(set(test.columns)))

Parse dates correctly

In [ ]:
train['date'] = pd.to_datetime(train['date'].apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:]))
test['date'] = pd.to_datetime(test['date'].apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:]))
train['visitStartTime'] = pd.to_datetime(train['visitStartTime'], unit='s')
test['visitStartTime'] = pd.to_datetime(test['visitStartTime'], unit='s')

In [ ]:
print(f'First date in train set is {train["date"].min()}. Last date in train set is {train["date"].max()}.')
print(f'First date in test set is {test["date"].min()}. Last date in test set is {test["date"].max()}.')

In [ ]:
print(f'First date in train set is {train["visitStartTime"].min()}. Last date in train set is {train["visitStartTime"].max()}.')
print(f'First date in test set is {test["visitStartTime"].min()}. Last date in test set is {test["visitStartTime"].max()}.')

The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.

You should *always* consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [ ]:
add_datepart(train, "date", drop=False)
add_datepart(test, "date", drop=False)
add_datepart(train, "visitStartTime", time=True, drop=False)
add_datepart(test, "visitStartTime", time=True, drop=False)

Next we'll fill in missing values to avoid complications with `NA`'s. `NA` (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary *signal value* that doesn't otherwise appear in the data.

In [ ]:
print ("think about it")

In [ ]:
train.to_feather(f'{PATH}train')
test.to_feather(f'{PATH}test')
#train = pd.read_feather(f'{PATH}train')
#test = pd.read_feather(f'{PATH}test')

## Durations

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
* Running averages
* Time until next event
* Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function `get_elapsed` for cumulative counting across a sorted dataframe. Given a particular field `fld` to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime na's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [ ]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

We'll be applying this to a subset of columns:

In [ ]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [ ]:
#df = train[columns]
df = train[columns].append(test[columns])

Let's walk through an example.

Say we're looking at School Holiday. We'll first sort by Store, then Date, and then call `add_elapsed('SchoolHoliday', 'After')`:
This will apply to each row with School Holiday:
* A applied to every row of the dataframe in order of store and date
* Will add to the dataframe the days since seeing a School Holiday
* If we sort in the other direction, this will count the days until another holiday.

In [ ]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We'll do this for two more fields.

In [ ]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [ ]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We're going to set the active index to Date.

In [ ]:
df = df.set_index("Date")

Then set null values from elapsed field calculations to 0.

In [ ]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [ ]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

Next we'll demonstrate window functions in pandas to calculate rolling quantities.

Here we're sorting by date (`sort_index()`) and counting the number of events of interest (`sum()`) defined in `columns` in the following week (`rolling()`), grouped by Store (`groupby()`). We do the same in the opposite direction.

In [ ]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [ ]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

Next we want to drop the Store indices grouped together in the window function.

Often in pandas, there is an option to do this in place. This is time and memory efficient when working with large datasets.

In [ ]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [ ]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [ ]:
df.reset_index(inplace=True)

Now we'll merge these values onto the df.

In [ ]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [ ]:
df.drop(columns,1,inplace=True)

In [ ]:
df.head()

It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [ ]:
df.to_feather(f'{PATH}df')

In [ ]:
df = pd.read_feather(f'{PATH}df')

In [ ]:
df["Date"] = pd.to_datetime(df.Date)

In [ ]:
df.columns

In [ ]:
joined = join_df(joined, df, ['Store', 'Date'])

In [ ]:
joined_test = join_df(joined_test, df, ['Store', 'Date'])

The authors also removed all instances where the store had zero sale / was closed. We speculate that this may have cost them a higher standing in the competition. One reason this may be the case is that a little exploratory data analysis reveals that there are often periods where stores are closed, typically for refurbishment. Before and after these periods, there are naturally spikes in sales that one might expect. By ommitting this data from their training, the authors gave up the ability to leverage information about these periods to predict this otherwise volatile behavior.

In [ ]:
joined = joined[joined.Sales!=0]

We'll back this up as well.

In [ ]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [ ]:
joined.to_feather(f'{PATH}joined')
joined_test.to_feather(f'{PATH}joined_test')

We now have our final set of engineered features.

While these steps were explicitly outlined in the paper, these are all fairly typical feature engineering steps for dealing with time series data and are practical in any similar setting.

## Create features

In [ ]:
train = pd.read_feather(f'{PATH}train')
test = pd.read_feather(f'{PATH}test')

In [ ]:
train.head().T.head(60)

Now that we've engineered all our features, we need to convert to input compatible with a neural network.

This includes converting categorical variables into contiguous integers or one-hot encodings, normalizing continuous features to standard normal, etc...

In [ ]:
contin_vars = ['visitNumber', 'totals.hits', 'totals.pageviews']
no_use = ['fullVisitorId', 'date', 'sessionId', 'visitId', 'visitStartTime', 'visitStartTimeElapsed', 
          'visitStartTimeYear', 'visitStartTimeMonth', 'visitStartTimeWeek', 'visitStartTimeDay', 'visitStartTimeDayofweek',
          'visitStartTimeDayofyear', 'visitStartTimeIs_month_end', 'visitStartTimeIs_month_start', 'visitStartTimeIs_quarter_end',
          'visitStartTimeIs_quarter_start', 'visitStartTimeIs_year_end', 'visitStartTimeIs_year_start', 
          'totals.transactionRevenue']
cat_vars = [col for col in train.columns if col not in contin_vars and col not in no_use]

In [ ]:
dep = 'totals.transactionRevenue'
train = train[cat_vars+contin_vars+[dep, 'date']].copy()

In [ ]:
test[dep] = 0
test = test[cat_vars+contin_vars+[dep, 'date', 'fullVisitorId']].copy()

In [ ]:
for v in cat_vars: train[v] = train[v].astype('category').cat.as_ordered()

In [ ]:
apply_cats(test, train)

In [ ]:
for v in contin_vars:
    train[v] = train[v].fillna(0).astype('float32')
    test[v] = test[v].fillna(0).astype('float32')

We're going to run on a sample.

In [ ]:
n = len(train)
idxs = get_cv_idxs(n, val_pct=150000/n)
train_samp = train.iloc[idxs].set_index("date")
samp_size = len(train_samp); samp_size

To run on the full dataset, use this instead:

In [ ]:
samp_size = n
train_samp = train.set_index("date")

We can now process our data...

In [ ]:
train_samp.head(2)

In [ ]:
df, y, nas, mapper = proc_df(train_samp, 'totals.transactionRevenue', do_scale=True)
yl = np.log(y)

In [ ]:
test = test.set_index("date")

In [ ]:
df_test, _, nas, mapper = proc_df(test, 'totals.transactionRevenue', do_scale=True, skip_flds=['fullVisitorId'],
                                  mapper=mapper, na_dict=nas)

In [ ]:
df.head(2)

In time series data, cross-validation is not random. Instead, our holdout data is generally the most recent data, as it would be in real application. This issue is discussed in detail in [this post](http://www.fast.ai/2017/11/13/validation-sets/) on our web site.

One approach is to take the last 25% of rows (sorted by date) as our validation set.

In [ ]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

An even better option for picking a validation set is using the exact same length of time period as the test set uses - this is implemented here:

In [ ]:
print(f'First date in train set is {df.index.min()}. Last date in train set is {df.index.max()}.')
print(f'First date in test set is {df_test.index.min()}. Last date in test set is {df_test.index.max()}.')

In [ ]:
val_idx = np.flatnonzero(
    (df.index<=datetime.datetime(2017,8,1)) & (df.index>=datetime.datetime(2017,4,1)))

In [ ]:
len(val_idx)

## DL

We're ready to put together our models.

Root-mean-squared percent error is the metric Kaggle used for this competition.

In [ ]:
def inv_y(a): return np.exp(a)

def exp_rmse(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

We can create a ModelData object directly from out data frame.

In [ ]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)

Some categorical variables have a lot more levels than others. Store, in particular, has over a thousand!

In [ ]:
cat_sz = [(c, len(train_samp[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

We use the *cardinality* of each variable (that is, its number of unique values) to decide how large to make its *embeddings*. Each level will be associated with a vector with length defined as below.

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
# m.summary()

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot(100)

### Sample

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.fit(lr, 3, metrics=[exp_rmse])

In [ ]:
m.fit(lr, 5, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
m.fit(lr, 2, metrics=[exp_rmspe], cycle_len=4)

### All

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.fit(lr, 1, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
m.fit(lr, 2, metrics=[exp_rmspe], cycle_len=4)

### Test

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-4

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
m.save('val0')

In [ ]:
m.load('val0')

In [ ]:
x,y=m.predict_with_targs()

In [ ]:
exp_rmspe(x,y)

In [ ]:
pred_test=m.predict(True)

In [ ]:
pred_test = np.exp(pred_test)

In [ ]:
joined_test['Sales']=pred_test

In [ ]:
csv_fn=f'{PATH}tmp/sub.csv'

In [ ]:
joined_test[['Id','Sales']].to_csv(csv_fn, index=False)

In [ ]:
FileLink(csv_fn)

## RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
((val,trn), (y_val,y_trn)) = split_by_idx(val_idx, df.values, yl)

In [ ]:
m = RandomForestRegressor(n_estimators=40, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)
m.fit(trn, y_trn);

In [ ]:
preds = m.predict(val)
m.score(trn, y_trn), m.score(val, y_val), m.oob_score_, exp_rmspe(preds, y_val)